In [1]:
%matplotlib inline
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def download(url, folder, sha1_hash=None):
    """Download a file to folder and return the local filepath."""

def extract(filename, folder):
    """Extract a zip/tar file into folder."""

In [3]:
class KaggleHouse(d2l.DataModule):
    def __init__(self, batch_size, train=None, val=None):
        super().__init__()
        self.save_hyperparameters()
        if self.train is None:
            self.raw_train = pd.read_csv(d2l.download(
                d2l.DATA_URL + 'kaggle_house_pred_train.csv', self.root,
                sha1_hash='585e9cc93e70b39160e7921475f9bcd7d31219ce'))
            self.raw_val = pd.read_csv(d2l.download(
                d2l.DATA_URL + 'kaggle_house_pred_test.csv', self.root,
                sha1_hash='fa19780a7b011d9b009e8bff8e99922a8ee2eb90'))

In [4]:
data = KaggleHouse(batch_size=64)
print(data.raw_train.shape)
print(data.raw_val.shape)

(1460, 81)
(1459, 80)


In [6]:
print(data.raw_train.iloc[:4, [0, 1, 2, 3, -3, -2, -1]])

   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000


In [7]:
@d2l.add_to_class(KaggleHouse)
def preprocess(self):
    # Remove the ID and label columns
    label = 'SalePrice'
    features = pd.concat(
        (self.raw_train.drop(columns=['Id', label]),
         self.raw_val.drop(columns=['Id'])))
    # Standardize numerical columns
    numeric_features = features.dtypes[features.dtypes!='object'].index
    features[numeric_features] = features[numeric_features].apply(
        lambda x: (x - x.mean()) / (x.std()))
    # Replace NAN numerical features by 0
    features[numeric_features] = features[numeric_features].fillna(0)
    # Replace discrete features by one-hot encoding
    features = pd.get_dummies(features, dummy_na=True)
    # Save preprocessed features
    self.train = features[:self.raw_train.shape[0]].copy()
    self.train[label] = self.raw_train[label]
    self.val = features[self.raw_train.shape[0]:].copy()

''' 
这段代码定义了一个名为`preprocess`的函数，用于预处理Kaggle房价预测任务的数据。该函数主要执行以下操作：

1. 删除ID和标签列：从原始训练集和验证集中删除ID列和'SalePrice'标签列，将剩余的特征组合在一起。

2. 标准化数值特征：找到数值型特征（即不是对象类型的特征），并对它们进行标准化。标准化是将每个特征减去其均值并除以其标准差，从而使特征具有零均值和单位方差。

3. 用0替换缺失的数值特征：将数值特征中的NaN值替换为0。

4. 用one-hot编码替换离散特征：将离散特征转换为one-hot编码表示。`pd.get_dummies`函数可以自动找到离散特征并创建one-hot编码。`dummy_na=True`表示对于具有NaN值的特征，也创建一个表示缺失值的新列。

5. 保存预处理后的特征：将预处理后的特征分为训练集和验证集，并将'SalePrice'标签列添加回训练集。

这个预处理函数的目的是为了在训练模型之前清洗和准备数据。通过标准化数值特征、处理缺失值和将离散特征转换为one-hot编码，我们可以确保模型接收到适当格式的输入数据。
'''

In [8]:
data.preprocess()
data.train.shape

(1460, 331)

In [9]:
@d2l.add_to_class(KaggleHouse)
def get_dataloader(self, train):
    label = 'SalePrice'
    data = self.train if train else self.val
    if label not in data: return
    get_tensor = lambda x: torch.tensor(x.values.astype(float),
                                      dtype=torch.float32)
    # Logarithm of prices
    tensors = (get_tensor(data.drop(columns=[label])),  # X
               torch.log(get_tensor(data[label])).reshape((-1, 1)))  # Y
    return self.get_tensorloader(tensors, train)

  ''' 
  这段代码定义了一个名为`get_dataloader`的函数，用于为Kaggle房价预测任务的数据生成数据加载器。该函数根据输入参数`train`的值，为训练集或验证集创建一个PyTorch数据加载器。

1. 首先，根据`train`参数选择要处理的数据集（训练集或验证集）。如果数据集中没有标签列（'SalePrice'），则返回None。

2. 定义一个名为`get_tensor`的辅助函数，用于将Pandas DataFrame数据转换为PyTorch张量。这个函数接受一个DataFrame对象，将其值转换为浮点数，并创建一个相应的PyTorch张量。

3. 使用`get_tensor`函数创建输入特征（X）和目标变量（Y）的张量。对于目标变量（房价），我们对其值取对数，以便在训练过程中更好地处理不同数量级的房价。将房价张量Y调整为列向量。

4. 调用`self.get_tensorloader`函数，为给定的输入特征和目标变量张量创建一个PyTorch数据加载器。这个加载器将用于在训练和评估模型时批量加载数据。

这个`get_dataloader`函数的目的是简化从预处理过的数据集中获取数据的过程。通过为训练集和验证集创建数据加载器，我们可以轻松地在训练和评估模型时批量加载数据。
'''